In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

if not torch.backends.mps.is_available():
    device = torch.device('cpu')
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    device = torch.device("mps")

# Basic block for ResNet
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

# ResNet model
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)  # 1 input channel
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2)
        self.linear = nn.Linear(8960 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


In [6]:
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

resize_scale = 0.5

def custom_transform(image):
    # Crop the image to the desired region of interest (ROI)
    image = image.crop((55, 35, 390, 253))
    # Convert the cropped image to a PyTorch tensor
    return transforms.ToTensor()(image)

def load_data(resize_images = False):
    # Define your data transformation (without resizing)
    transforms_list = []
    transforms_list.append(transforms.Grayscale(num_output_channels=1))
    transforms_list.append(transforms.Lambda(custom_transform))  # Apply the custom transformation
    if resize_images:
        transforms_list.append(transforms.Resize((int(218*resize_scale), int(335*resize_scale))))
    transforms_list.append(transforms.Normalize((0.5,), (0.5,)))  # Normalize to [-1, 1]
    data_transform = transforms.Compose(transforms_list)

    class CustomImageDataset(datasets.ImageFolder):
        def __init__(self, root, transform=None):
            super(CustomImageDataset, self).__init__(root=root, transform=transform)

    # Define the path to your data folder
    data_dir = 'data/images_original'

    # Create an instance of your custom dataset
    custom_dataset = CustomImageDataset(root=data_dir, transform=data_transform)

    # Calculate the size of the training and testing sets
    total_size = len(custom_dataset)
    train_size = int(0.8 * total_size)
    test_size = total_size - train_size

    # Split the dataset
    train_dataset, test_dataset = random_split(custom_dataset, [train_size, test_size])
    batch_size = 10  # You can adjust this batch size as needed

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

train_loader, test_loader = load_data(resize_images=False)

# Get a batch of data from the training loader
data_iterator = iter(train_loader)
images, labels = next(data_iterator)

print("Image shape:", images[0].shape)
num_channels = images[0].shape[0]  # The number of channels in the image
print("Number of channels:", num_channels)

Image shape: torch.Size([1, 218, 335])
Number of channels: 1


In [7]:
import torch.nn as nn
import torch.optim as optim

# Define your neural network
net = ResNet18()
net.to(device)

# Define the loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.SGD(net.parameters(), lr=0.0003, momentum=0.9)

# Set the number of training epochs
num_epochs = 25

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0

    # Iterate over the training dataset
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()

    # Evaluate the model on the test dataset and calculate accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1} | Loss: {running_loss:.3f} | Accuracy: {test_accuracy:.2f}%")
    running_loss = 0.0

print("Training finished")

# Save the trained model if desired
# torch.save(net.state_dict(), "my_model.pth")


Epoch 1 | Loss: 192.194 | Accuracy: 33.00%
Epoch 2 | Loss: 131.885 | Accuracy: 36.00%
Epoch 3 | Loss: 115.725 | Accuracy: 47.50%
Epoch 4 | Loss: 79.538 | Accuracy: 54.50%
Epoch 5 | Loss: 71.702 | Accuracy: 59.00%
Epoch 6 | Loss: 56.326 | Accuracy: 59.00%
Epoch 7 | Loss: 57.749 | Accuracy: 53.00%
Epoch 8 | Loss: 37.454 | Accuracy: 62.00%
Epoch 9 | Loss: 24.690 | Accuracy: 64.00%
Epoch 10 | Loss: 21.647 | Accuracy: 67.50%
Epoch 11 | Loss: 13.959 | Accuracy: 62.00%
Epoch 12 | Loss: 10.079 | Accuracy: 66.50%
Epoch 13 | Loss: 6.870 | Accuracy: 65.00%
Epoch 14 | Loss: 5.986 | Accuracy: 67.00%
Epoch 15 | Loss: 4.197 | Accuracy: 65.00%
Epoch 16 | Loss: 5.403 | Accuracy: 64.00%
Epoch 17 | Loss: 3.599 | Accuracy: 68.00%
Epoch 18 | Loss: 2.678 | Accuracy: 63.50%
Epoch 19 | Loss: 2.276 | Accuracy: 67.50%
Epoch 20 | Loss: 2.736 | Accuracy: 67.50%
Epoch 21 | Loss: 2.206 | Accuracy: 68.50%
Epoch 22 | Loss: 2.845 | Accuracy: 67.50%
Epoch 23 | Loss: 2.021 | Accuracy: 64.50%
Epoch 24 | Loss: 1.874 | Acc

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
import matplotlib.pyplot as plt
import random

# Define your neural network
net = ResNet18()
net.to(device)

# Define the loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-6, momentum=0.9)

# Define a learning rate range
start_lr = 1e-7  # Lower starting learning rate
end_lr = 1.0  # Wider range
num_steps = 10  # Fewer steps

# Create a learning rate scheduler with logarithmic interpolation
lr_lambda = lambda step: 10 ** (start_lr + (end_lr - start_lr) * step / (num_steps - 1))
scheduler = LambdaLR(optimizer, lr_lambda)

# Lists to store learning rates and losses
learning_rates = []
losses = []

# Set the number of training iterations for each learning rate step
num_iterations = 20  # A small number of minibatches

# Counter to keep track of the current step
step_counter = 0

# Learning rate range test loop
for step in range(num_steps):
    # Set the learning rate using the scheduler
    current_lr = optimizer.param_groups[0]['lr']
    learning_rates.append(current_lr)

    running_loss = 0.0

    # Create a random subset from the training dataset
    random_subset = random.sample(range(len(train_loader.dataset)), num_iterations * train_loader.batch_size)

    for i in random_subset:
        inputs, labels = train_loader.dataset[i]  # Assuming a tuple structure

        # Convert inputs and labels to tensors and move to device
        inputs = torch.tensor(inputs).to(device)
        labels = torch.tensor(labels).to(device)

        optimizer.zero_grad()
        outputs = net(inputs.unsqueeze(0))
        loss = criterion(outputs, labels.unsqueeze(0))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Record the loss for this step
    losses.append(running_loss / (num_iterations * train_loader.batch_size))

    # Increment the step counter
    step_counter += 1

    # Update the learning rate
    scheduler.step()

    print(f"Step {step_counter}/{num_steps} | Learning Rate: {current_lr} | Loss: {running_loss / (num_iterations * train_loader.batch_size)}")

# Plot the learning rate vs. loss curve
plt.plot(learning_rates, losses)
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Loss')
plt.show()

# Find the optimal learning rate
optimal_lr = learning_rates[losses.index(min(losses))]
print(f"Optimal Learning Rate: {optimal_lr}")

# Now you can use the optimal learning rate for training
# Update the learning rate in your optimizer with the found optimal_lr
# optimizer = optim.SGD(net.parameters(), lr=optimal_lr, momentum=0.9)

# # Continue with your training loop using the optimal learning rate
# for epoch in range(1):  # You can set the number of full training epochs here
#     # Rest of your training code
#     # ...

# print("Training finished")


NameError: name 'ResNet18' is not defined

In [21]:
import math

start_lr = 1e-7  # Lower starting learning rate
end_lr = 1  # Wider range
num_steps = 10  # Fewer steps

# Calculate the scaling factor for each step
scaling_factor = math.exp(math.log(end_lr / start_lr) / (num_steps - 1))

lr_lambda = lambda step: start_lr * (scaling_factor ** step)

for step in range(num_steps):
    print(lr_lambda(step))

1e-07
5.99484250318941e-07
3.593813663804627e-06
2.154434690031884e-05
0.0001291549665014884
0.0007742636826811272
0.00464158883361278
0.02782559402207125
0.16681005372000593
1.0000000000000002
